In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
df_gbm = pd.read_csv('C:\\Users\\Praveen Palem\\Documents\\Knime\\Data\\GBM_predictions.csv')
df_gbm.head()

,VMail Message,CustServ Calls,Day Calls,Day Charge,Eve Calls,Eve Charge,Night Calls,Night Charge,Intl Calls,Intl Charge,Phone,City,Account Length,Churn,Int'l Plan,VMail Plan,P (Churn=0),P (Churn=1),Prediction (Churn)
0,-0.591671,1.092477,-0.619642,-0.425761,-1.260592,0.920150,0.761030,-0.171066,0.211502,0.736852,394-8006,Cedarville,-0.152297,0,0,0,9.903272e-01,0.009673,0
1,-0.591671,1.092477,1.024678,0.591580,0.747175,-3.348331,-1.129754,1.828167,2.243020,0.842984,383-1121,Lomita,1.781302,0,0,0,9.996933e-01,0.000307,0
2,1.599971,-1.188040,1.373474,-0.614758,-1.009622,-0.759404,0.454416,-1.414545,0.617806,1.532848,341-9764,Lomita,-1.633887,0,0,1,9.993207e-01,0.000679,0
3,1.819135,1.092477,0.277260,0.442542,-0.055932,-0.262962,0.352211,-1.959391,0.617806,0.100055,402-1381,Fresno,0.877281,0,1,1,4.747917e-75,1.000000,1
4,-0.591671,1.092477,0.077949,-0.981951,-0.758651,-0.673571,1.783075,-0.258945,-1.007408,-0.297943,372-9976,Hamilton City,-0.076962,0,0,0,9.869582e-01,0.013042,0


In [3]:
idx = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
df_sub = df_gbm[['Churn','P (Churn=1)']]
for i in idx:
    df_sub['Pred_' + str(i)] = np.where(df_sub['P (Churn=1)'] >= i, 1, 0) 
df_sub.head()

C:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Churn,P (Churn=1),Pred_0,Pred_0.1,Pred_0.2,Pred_0.3,Pred_0.4,Pred_0.5,Pred_0.6,Pred_0.7,Pred_0.8,Pred_0.9,Pred_1
0,0,0.009673,1,0,0,0,0,0,0,0,0,0,0
1,0,0.000307,1,0,0,0,0,0,0,0,0,0,0
2,0,0.000679,1,0,0,0,0,0,0,0,0,0,0
3,0,1.000000,1,1,1,1,1,1,1,1,1,1,1
4,0,0.013042,1,0,0,0,0,0,0,0,0,0,0


In [4]:

for i in idx:
    df_sub['TN' + str(i)] = np.where((df_sub['Pred_' + str(i)] == 0) & (df_sub['Churn'] == 0), 1, 0)
    df_sub['FP' + str(i)] = np.where((df_sub['Pred_' + str(i)] == 1) & (df_sub['Churn'] == 0), 1, 0)
    df_sub['FN' + str(i)] = np.where((df_sub['Pred_' + str(i)] == 0) & (df_sub['Churn'] == 1), 1, 0)
    df_sub['TP' + str(i)] = np.where((df_sub['Pred_' + str(i)] == 1) & (df_sub['Churn'] == 1), 1, 0)

C:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [5]:
df_cf = df_sub.sum(axis = 0)
#df_cf
tp_bnft = 1500
fn_cost = -2500
fp_cost = -200
tn_bnft = 100

def compute_benefit (tp, fn, fp, tn) -> float:
    return 0.8*tp*tp_bnft + (1-0.8)*tp*fn_cost + fn*fn_cost + tn*tn_bnft + 0.90*fp*fp_cost + (1-0.9)*fp*tn_bnft

In [6]:
df_res = pd.DataFrame(index = idx, columns=['benefit'])
print ('i   tp   fn   fp   tn')
for i in idx:
    tp =  df_cf['TP' + str(i)]
    fn =  df_cf['FN' + str(i)]
    fp =  df_cf['FP' + str(i)]
    tn =  df_cf['TN' + str(i)]
    print (float(i), tp,fn,fp,tn)
    df_res.at[i,'benefit'] = compute_benefit(tp, fn, fp, tn)

i   tp   fn   fp   tn
0.0 97.0 0.0 569.0 1.0
0.1 80.0 17.0 88.0 482.0
0.2 79.0 18.0 70.0 500.0
0.3 79.0 18.0 61.0 509.0
0.4 74.0 23.0 55.0 515.0
0.5 71.0 26.0 52.0 518.0
0.6 67.0 30.0 51.0 519.0
0.7 61.0 36.0 49.0 521.0
0.8 55.0 42.0 47.0 523.0
0.9 50.0 47.0 43.0 527.0
1.0 30.0 67.0 39.0 531.0


In [7]:
df_res

,benefit
0.0,-28730
0.1,46740
0.2,48400
0.3,50830
0.4,36450
0.5,27660
0.6,15130
0.7,-3530
0.8,-22190
0.9,-37110
